In [1]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
from urllib.request import urlopen
import numpy as np
import ssl

## DBMS 코드에서 연결 사용
- DBMS에 맞는 패키지가 설치되어야 함
- MySQL (MariaDB)은 pymysql 패키지 필요(설치해야 되는 패키지

In [2]:
# !pip install pymysql

In [3]:
import pymysql

### pymysql 사용법
1. dbms 연결
    - pymysql.connect(host='서버주소(ip주소)', port=3306, user='userid', password='userpass', charset='utf-8')
    - port : 서버주소를 이용해서 서버컴퓨터의 입구까지 연결을 했을 때
        - port를 이용해서 각 프로그램의 방으로 연결하는 개념
<hr>
2. 1번에서 연결한 정보(객체)를 이용해서 cursor 객체 생성
    - connect.cursor()
    - cursor 객체 : DBMS와 통로 역할을 함
        - slq문의 명령을 dbms로 전달
        - 실행된 sql문의 결과 반환
<hr>
3. sql 문 작성
    - 저장소 관련 sql 문 : create, alter, drop 으로 시작되는 sql문
    - 데이터 저장과 관련 sql 문 : table 데이터 삽입 삭제 갱신과 관련
        - insert, delete, update로 시작되는 sql 문
    - 검색 sql문 : select로 시작되는 sql문(반환데이터가 있음)
<hr>
4. cursor 객체를 이용해서 3번 작성한 sql 문을 실행
    - cursor.execute(sql문)
<hr>  
5. 검색과 관련된 sql 문인 경우
    - cursor.fetchXXX() 를 이용해서 데이터 반환
    - cursor.fetchall() : 반환된 모든 데이터 한번에 전달받기
    - cursor.fetchone() : 반환된 모든 데이터를 한개씩 전달받기

In [4]:
# connect 객체 생성
db = pymysql.connect(host='localhost', port=3306, user='root', password='mjh123456')
# cursor 객체
cursor = db.cursor()

In [5]:
sql = 'drop database beauty_shop'  # beauty_shop db 삭제할 것
# cursor.execute(sql)

# db에 없어서 삭제할 수 없다 - 에러발생
# OperationalError: (1008, "Can't drop database 'beauty_shop'; database doesn't exist")

### DBMS 구조
- 제일 넓은 구조 : database
    - 생성 : create database db명 단, db명은 중복되면 안된다.
    - 삭제 : drop database db명 단, db명은 기존에 생성되어있어야 함
<hr>
- database 내에 실제 data 저장 구조 : table
    - 생성 : create table
    - 삭제 : drop table
    - 수정 : alter table


In [6]:
## db 생성
sql = 'create database beauty_shop'
cursor.execute(sql)

1

In [7]:
## 생성된 db 확인
sql = 'show databases'
cursor.execute(sql)
result = cursor.fetchall() # 결과는 DBMS의 db명 모두 반환
result

(('beauty_shop',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sys',))

In [8]:
## 사용할 db를 결정해서 명령  : 바꿔주세요
## use db명
sql = 'use beauty_shop'
cursor.execute(sql)

0

In [9]:
# 현재 사용 db 반환  : 조회해주세요
sql = 'select database()' # 현재 설정된 db명 반환
cursor.execute(sql) # dbms로 sql 명령문 전달 후 결과 반환
result = cursor.fetchone()
result

('beauty_shop',)

In [10]:
# 쇼핑몰 크롤링 data를 저장할 table 생성(dbms에 생성 => sql 문 사용)
sql = '''
    CREATE TABLE product (
        PRODUCT_CODE int AUTO_INCREMENT NOT NULL,
        TITLE VARCHAR(200) NOT NULL,
        ORI_PRICE FLOAT,
        DISCOUNT_PRICE FLOAT,
        link VARCHAR(200),
        PRIMARY KEY(PRODUCT_CODE)
    );
'''

 CREATE TABLE 테이블명 (<br><br>
        &nbsp;&nbsp;&nbsp;컬럼명, 컬럼data형태, 옵션 빈칸허가여부,<br><br>
        &nbsp;&nbsp;&nbsp;PRODUCT_CODE int AUTO_INCREMENT NOT NULL,<br>
        &nbsp;&nbsp;&nbsp;TITLE VARCHAR(200) NOT NULL,<br>
        &nbsp;&nbsp;&nbsp;ORI_PRICE FLOAT,<br>
        &nbsp;&nbsp;&nbsp;DISCOUNT_PRICE FLOAT,<br>
        &nbsp;&nbsp;&nbsp;link VARCHAR(200),<br>
        &nbsp;&nbsp;&nbsp;PRIMARY KEY(PRODUCT_CODE)<br>
    );

- AUTO_INCREMENT : 0부터 시작하고 1씩 증가하는 값을 dbms 가 자동으로 생성해줌
    - 값을 넣지 않는다
    - 행 인덱스로 사용(유일한 값이 필요할 경우 사용)    
- not null : 결측치 허용하지 않음(이 속성 없으면 결측치여도 됨)
- VARCHAR(200) : 가변 문자(최대 문자 200문자내에서 사용할 수 있음)
- PRIMARY KEY(PRODUCT_CODE) : 크롤링한 화장품 정보를 유일하게 구별 할 수 있는 컬럼

In [11]:
# 테이블 생성 sql문(쿼리문) 실행하기
cursor.execute(sql)

# 외부에서 연결해서 사용할 경우 dbms(maria db)는 연결을 지속시킬 수 없기 때문에 
# 명령을 클라이언트 단에서 먼저 진행하고
# 나중에 dbms에 반영한다.(create table, insert 같은 명령들)
# 테이블 같은 경우 생성하고 바로 명시적 반영 하는게 좋음
# connect객체 함수 commit()을 사용
db.commit()  # dbms에 완전 반영

In [12]:
## 생성된 db 확인
sql = 'show tables'
cursor.execute(sql)
result = cursor.fetchall() # 결과는 DBMS의 db명 모두 반환
result

(('product',),)

In [13]:
sql = 'desc product' # desc 테이블명 : 테이블 컬럼 구조 반환
cursor.execute(sql)
result = cursor.fetchall() # 여러 컬럼 이므로 fetchall() 사용
result

(('PRODUCT_CODE', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('TITLE', 'varchar(200)', 'NO', '', None, ''),
 ('ORI_PRICE', 'float', 'YES', '', None, ''),
 ('DISCOUNT_PRICE', 'float', 'YES', '', None, ''),
 ('link', 'varchar(200)', 'YES', '', None, ''))

### 제품 1개에 대한 정보 추출 후 db table에 저장

In [19]:
# df 생성
prd_dict = {'품목':[], '가격':[], '세일가격':[], '제품경로':[]}
prd_df = pd.DataFrame(prd_dict)
prd_df

,품목,가격,세일가격,제품경로


In [18]:
# 각 제품 정보 추출하는 함수
# box 는 class 값이 description인 div 태그 1개가 전달
def get_product_info(box):
    try:
        strong = box.find('strong', {'class':'name'})
        title = strong.text.split(':')[1].strip()
        link = strong.a['href']
        lis = box.find('ul').findAll('li')
        price = (lis[0].text.split(' ')[2])
        sale_price = (lis[1].text.split(' ')[2])    
        return {'품목':title, '가격':price, '세일가격':sale_price, '제품경로':link,}
    except:
        print('error')

In [24]:
def get_page_product(url):
    global prd_df 
    # global 변수를 함수 내부가 아닌 외부에서 찾을 것이라는 의미
    # global은 전역변수(함수내/외부에서 모두 사용)로 생성
    bs_obj = get_request_product(url)
    # 페이지 내에 전체 제품 정보 추출
    boxes = bs_obj.findAll('div',{'class': 'description'})
    # 각 제품에 대한 정보 추출 후 df에 저장
    for box in boxes[2:]:
        prd = get_product_info(box)
        save_data(prd)  # db에 저장하는 함수 호출(사용자 정의 함수 = 해당 함수 만들 예정)
#         res = pd.DataFrame(get_product_info(box), index=range(1,2))
#         prd_df = pd.concat([prd_df,res], axis=0, ignore_index=True)

### db table에 data 저장
- Insert into 구문 사용
- insert into 테이블명( 컬럼명1, 컬럼명2,...,컬럼명n) values (컬럼값1, 컬럼값2,...,컬럼값n)
- 문자열값은 ''로 표현
- insert into <b>product</b>(title,ori_price,discount_price, link) <b>values</b>('toner', 22.0, 17.0, 'http://a.b.c')

In [ ]:
# insert into product(title, ori_price, discount_price, link) values('SKINFOOD Yuja C Dark Spot Clear Toner 200ml',39.00,35.10,'/product/skinfood-yuja-c-dark-spot-clear-toner-200ml/62935/category/1019/display/1/')

In [38]:
# insert 구문을 이용해서 사용자 정의 함수 만들기 => save_data()
# prd_info 라는 제품 1개의 정보가 딕셔너리로 전달 됨
def save_data(prd_info):
    sql = "insert into product(title, ori_price, discount_price, link) values('" \
    + prd_info['품목'] + "'," \
    + prd_info['가격'] + "," \
    + prd_info['세일가격'] + ",'" \
    + prd_info['제품경로'] + "')"
    print(sql)
    cursor.execute(sql)

In [16]:
def get_request_product(url):
    # 요청 후 코드 추출
    url = url
    context = ssl._create_unverified_context()
    htmls = urlopen(url, context=context)
    htmls = htmls.read()
    # bs 객체 생성
    bs_obj = BeautifulSoup(htmls, 'html.parser')
    return bs_obj

In [21]:
url = 'https://jolse.com/category/toners-mists/1019?page=4'

In [40]:
# db 저장 진행 후 확인 
get_page_product(url)

insert into product(title, ori_price, discount_price, link) values('SKINFOOD Yuja C Dark Spot Clear Toner 200ml',39.00,35.10,'/product/skinfood-yuja-c-dark-spot-clear-toner-200ml/62935/category/1019/display/1/')
insert into product(title, ori_price, discount_price, link) values('MISSHA Bee Pollen Renew Mist Ampoule 100ml',28.00,25.20,'/product/missha-bee-pollen-renew-mist-ampoule-100ml/62813/category/1019/display/1/')
insert into product(title, ori_price, discount_price, link) values('SOME BY MI BETA PANTHENOL REPAIR TONER 150ml',22.50,20.25,'/product/some-by-mi-beta-panthenol-repair-toner-150ml/62528/category/1019/display/1/')
insert into product(title, ori_price, discount_price, link) values('SOME BY MI BETA PANTHENOL REPAIR SERUM 30ml',28.10,25.29,'/product/some-by-mi-beta-panthenol-repair-serum-30ml/62527/category/1019/display/1/')
insert into product(title, ori_price, discount_price, link) values('THANK YOU FARMER Phyto Relieful™ Cica Toner Pad 200ml (100pads)',33.00,29.70,'/produ

In [41]:
db.commit() # dbms 반영

In [42]:
# db에서 data 조회 후 확인
sql = 'select * from product' # product 테이블의 모든(*) data 반환
cursor.execute(sql)
result = cursor.fetchall()
result

((1,
  'SKINFOOD Yuja C Dark Spot Clear Toner 200ml',
  39.0,
  35.1,
  '/product/skinfood-yuja-c-dark-spot-clear-toner-200ml/62935/category/1019/display/1/'),
 (2,
  'MISSHA Bee Pollen Renew Mist Ampoule 100ml',
  28.0,
  25.2,
  '/product/missha-bee-pollen-renew-mist-ampoule-100ml/62813/category/1019/display/1/'),
 (3,
  'SOME BY MI BETA PANTHENOL REPAIR TONER 150ml',
  22.5,
  20.25,
  '/product/some-by-mi-beta-panthenol-repair-toner-150ml/62528/category/1019/display/1/'),
 (4,
  'SOME BY MI BETA PANTHENOL REPAIR SERUM 30ml',
  28.1,
  25.29,
  '/product/some-by-mi-beta-panthenol-repair-serum-30ml/62527/category/1019/display/1/'),
 (5,
  'THANK YOU FARMER Phyto Relieful™ Cica Toner Pad 200ml (100pads)',
  33.0,
  29.7,
  '/product/thank-you-farmer-phyto-relieful-cica-toner-pad-200ml-100pads/62414/category/1019/display/1/'),
 (6,
  'numbuzin No.1 Centella Re-Leaf Green Toner Pad 190ml (70pad)',
  25.0,
  22.5,
  '/product/numbuzin-no1-centella-re-leaf-green-toner-pad-190ml-70pad/6207